<!-- Banner Image -->


<!-- Links -->
<center>
  <a href="https://arxiv.org/abs/2306.17519" style="color: #06b6d4;">Paper</a> •
  <a href="https://kdf-workshop.github.io/kdf23/shared_task" style="color: #06b6d4;">Competition</a>
</center>

# Learning To Retrieve Prompts for In-Context Learning 🤙

Welcome!

In this notebook and tutorial, we showcase our KDF SIGIR 2023 shared task solution which won 3rd rank overall.

# Problem Statement
To predict relation between given entities in sentence. The data is specific to Finance Domain.
<center>
<img src="https://refind-re.github.io/dist/pic/Intro.png" width="80%">
</center>


# Data
<center>
<img src="https://refind-re.github.io/plotting/barplot.png" width="60%">
</center>
# Labels
<center>
<img src="https://i.ibb.co/kMwymsy/Screenshot-2024-01-23-at-11-47-11-PM.png" width="60%">
</center>

# Our Solution
<center>
<img src="https://i.ibb.co/tQ0jGP3/Screenshot-2024-01-23-at-11-50-31-PM.png" width="80%">
</center>


In [ ]:
!pip install torch==1.10.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html;

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.1+cu113 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2

In [ ]:
from random import shuffle
import pickle
import copy
from random import shuffle
ins_dic ={"ORG-ORG": "org:org:agreement_with	when org enter or plan to enter into an agreement with org Y for matters like settlement\n  license etc\n  org:org:shares_of	when an org holds some shares of another org or it invests in the other org through IPO\n  stocks\n voting rights etc\n  org:org:acquired_by	when an org is acquired by another org\n  org:org:subsidiary_of	when an org is subsidiary of another org","PERSON-UNIV": "pers:univ:attended	when a person attended a university as a student","PERSON-ORG": "pers:org:employee_of	when a person is an employee of an organization\n  pers:org:member_of	when a person is a member of an organization", "PERSON-TITLE": "pers:title:title	when a person has a title designation", "ORG-GPE": "org:gpe:formed_in	when an organization is formed on a certain date\n  org:gpe:headquartered_in	when organization is headquartered in a location\n  org:gpe:operations_in	when an organization has a office at a location or operates in a location", "ORG-MONEY": "org:money:revenue_of	when an organization reports revenue of given amount\n  org:money:profit_of	when an organization reports profit of given amount\n  org:money:loss_of	when an organization reports loss of given amount\n  org:money:cost_of	when an organization has a costing of given amount", "ORG-DATE": "org:date:formed_on	when an organization was formed on a certain date\n  org:date:acquired_on	when organization was aquired on a date\n  pers:univ:employee_of	when a person is an employee of a university", "PERSON-ORG":"pers:org:founder_of	when a person is a founder of an organization", "PERSON-GOV_AGY": "pers:gov_agy:member_of	when a person is an employee of a government agency"}


handle = open('train_kv_map.pickle', 'rb')
train_kv_map = pickle.load(handle)
handle2 = open('random_kv_map.pickle', 'rb')
random_kv_map = pickle.load(handle2)

def add_random_examples(rel_grp_name,n):
  results = []
  result_p_list = []
  dic_value = {}
  all_possible_example_classes = random_kv_map[rel_grp_name]
  rel_grp_name  = rel_grp_name.replace("PERSON","pers")
  shuffle(all_possible_example_classes)
  for pos in all_possible_example_classes:
    if pos[0] in dic_value:
      if len(dic_value[pos[0]])==n:
        continue
      dic_value[pos[0]] = dic_value[pos[0]] +[pos[1]]
    else:
      dic_value[pos[0]] = [pos[1]]
  for k,v in dic_value.items():
    results = results+v
  final_st_prompt = ""
  for r1 in results:
    li2 = copy.deepcopy(r1)
    del li2["sentence"]
    del li2["relation"]
    temp_p =  "\n\n"+r1["sentence"]+"\n"+str(li2)+"\nAnswer: "+r1["relation"]
    result_p_list.append(temp_p)

  return result_p_list
def post_process_result(rel_grp_name):
  rel_grp_name  = rel_grp_name.replace("PERSON","pers")
  all_correct_results = []
  rel1,rel2 = rel_grp_name.lower().split("-")
  data = json.load(open("pred.json"))
  prompt_temp = ""
  raw_prompt = data[0]["prompt"]
  all_prompt_examples = (raw_prompt.split("\n"))
  for pe in all_prompt_examples:
    flag_pe = False
    key_to_check = pe.split("Can we say")[0].strip()
    if key_to_check not in train_kv_map:
      continue
    ans_dict = ""
    for entry_i in train_kv_map[key_to_check]:
      # print(entry_i["rel_group"],rel_grp_name,"CHECKKKKK")
      if entry_i["rel_group"].replace("PERSON","pers") == rel_grp_name:
        li3 = copy.deepcopy(entry_i)
        del li3["relation"]
        ans_dict = str(li3)
    if rel1+":"+rel2 in pe:
      parsed_sen = key_to_check+"\n"+ans_dict+"\nAnswer: "+pe.split(" ")[-1]
      flag_pe = True
    if rel2+":"+rel1 in pe:
      parsed_sen = key_to_check+"\n"+ans_dict+"\nAnswer: "+pe.split(" ")[-1]
      flag_pe = True
    # if "no_relation" in pe:
    #   parsed_sen = key_to_check+"\n"+ans_dict+"\nAnswer: "+"no_relation"
    #   flag_pe = True
    if flag_pe:
      # parsed_sen = pe.split("Can we say")[0].strip()
      all_correct_results.append(parsed_sen)
  return (all_correct_results)
  # i = data[0]
def generate_ins(rel_grp_name):
  base_i = "INSTRUCTION	I am a knowledgable person. I will solve the relation extraction task. Given the context and two entities, i will first consider whether the most precise relation between two entities belong to any following class, if yes i will output the most precise relation, otherwise output will be no_relation"
  base_i =base_i +"\n"+ins_dic[rel_grp_name]
  return base_i

def generate_final_prompt():
  res = post_process_result(test_data["rel_group"])
  random1 =add_random_examples(test_data["rel_group"],5)
  all_examples = res+random1
  li4 = copy.deepcopy(test_data)
  del li4["sentence"]
  del li4["relation"]
  pmt = "\n".join(all_examples) + "\n\n"+test_data["sentence"]+"\n"+str(li4)+"\nAnswer: "
  ins = generate_ins(test_data["rel_group"])
  return pmt, ins



In [ ]:
import json
test_data = {"sentence": "Pursuant to the Merger Agreement , UnitedHealth Group has agreed to acquire all of the outstanding shares of Surgical Care Affiliates , Inc. s common stock for $ 57.00 per share , to be funded with a combination of cash and UnitedHealth Group common stock , as set forth in the Merger Agreement .", "rel_group": "ORG-ORG", "relation": "org:org:acquired_by", "ORG_1": "Surgical Care Affiliates , Inc. s", "ORG_2": "UnitedHealth Group"}
f1 = open ("test_kdf.json","w")
f1.write(json.dumps(test_data))
f1.close()

In [ ]:
!sh scripts/run_epr.sh

/content/icl-ceil/dense_retriever_new.py:246: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="configs", config_name="dense_retriever")
[2024-01-23 17:27:27,600][__main__][INFO] - {'output_file': 'output/epr/mrpc/EleutherAI/gpt-neo-2.7B/bert-fix_ctx-shared-bs64/train_retrieved.json', 'num_candidates': 1, 'num_ice': 50, 'task_name': 'mrpc', 'batch_size': 64, 'model_name': 'bert-base-uncased', 'faiss_index': 'output/epr/mrpc/EleutherAI/gpt-neo-2.7B/bert-fix_ctx-shared-bs64/index', 'pretrained_model_path': 'output/epr/mrpc/EleutherAI/gpt-neo-2.7B/bert-fix_ctx-shared-bs64', 'dpp_search': True, 'dpp_topk': 100, 'mode': 'cand_random', 'dataset_reader': {'_target_': 'src.dataset_readers.base_dsr.BaseDatasetReader', 'task_name': '${task_name}', 'model_name': '${model_name}', 'field': 'q', 'dataset_split': 'test', 'dataset_path': None, 'ds_size': None}, 'index_reader'

In [ ]:
api_key1 = "YOUR_OPENAI_KEY_HERE"
from openai import OpenAI
def chat(text,ins):
  client = OpenAI(api_key =api_key1 )

  completion = client.chat.completions.create(
    #model="gpt-4-1106-preview",
    #model="gpt-3.5-turbo",
    model="gpt-4",
    messages=[
    {"role": "system", "content": ins},
    {"role": "user", "content": text}
    ])
  return (completion.choices[0].message)

In [ ]:
icl_prompt, instruction = generate_final_prompt()

test_data1= copy.deepcopy(test_data)
del test_data1["sentence"]
del test_data1["relation"]
print(instruction)
print(icl_prompt)
print("##########")
print(chat(test_prompt,instruction))
print("##########")
print(chat(icl_prompt,instruction))

INSTRUCTION	I am a knowledgable person. I will solve the relation extraction task. Given the context and two entities, i will first consider whether the most precise relation between two entities belong to any following class, if yes i will output the most precise relation, otherwise output will be no_relation
org:org:agreement_with	when org enter or plan to enter into an agreement with org Y for matters like settlement
  license etc
  org:org:shares_of	when an org holds some shares of another org or it invests in the other org through IPO
  stocks
 voting rights etc
  org:org:acquired_by	when an org is acquired by another org
  org:org:subsidiary_of	when an org is subsidiary of another org
Effective September 29 , 2017 , ALLTEMP , INC . entered into a Share Exchange Agreement with Spider Investments , LLC , a company controlled by the former controlling shareholder of ALLTEMP , INC . prior to the Merger , which provided for Spider Investments , LLC to acquire all of the issued and out

In [ ]:
"""
Org:Org:Acquired_By
{"sentence": "Pursuant to the Merger Agreement , UnitedHealth Group has agreed to acquire all of the outstanding shares of Surgical Care Affiliates , Inc. s common stock for $ 57.00 per share , to be funded with a combination of cash and UnitedHealth Group common stock , as set forth in the Merger Agreement .", "rel_group": "ORG-ORG", "relation": "org:org:acquired_by", "ORG_1": "Surgical Care Affiliates , Inc. s", "ORG_2": "UnitedHealth Group"}
Org:GPE:Headquartered_In
{"sentence": "The peer group consists of all publicly traded Commercial Banks and Thrifts headquartered in New Jersey , New York , Ohio and Pennsylvania with total assets of $ 750 million to $ 3.5 billion as of the end of the four consecutive calendar quarters ending with the third quarter of each calendar year following the Award Date .", "rel_group": "ORG-GPE", "relation": "org:gpe:headquartered_in", "ORG": "Commercial Banks and Thrifts", "GPE": "Pennsylvania"}
Person:Organization:Member_Of
{"sentence": "Ms. Lee serves on the board of directors of the Alvin Ailey American Dance Theater and the Paley Center .", "rel_group": "PERSON-ORG", "relation": "pers:org:member_of", "PERSON": "Lee", "ORG": "the Alvin Ailey American Dance Theater"}
No Relation
{"sentence": "On July 10 , 2017 , Lead Plaintiff filed an amended federal securities class action complaint ( the Amended Complaint ) against Rentech , Inc , Keith Forman , and Jeffrey Spain .", "rel_group": "PERSON-ORG", "relation": "no_relation", "PERSON": "Jeffrey Spain", "ORG": "Rentech , Inc"}
"""
